In [10]:
import pandas as pd
import networkx as nx
import numpy as np
import json
from collections import defaultdict
import utils
from tqdm.notebook import tqdm, tqdm_notebook
tqdm_notebook.pandas()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
for tournament in tqdm(utils.TOURNAMENTS, total= len(utils.TOURNAMENTS)):   
    print("TOURNAMENT: %s" %tournament)

    events = pd.read_json('events\events_%s.json' %tournament)
    matches = pd.read_json('matches\matches_%s.json' %tournament)
    teams = pd.read_json('teams.json')

    #tira penalidades
    events = events[events['matchPeriod'] != 'P']

    # adaptando coluna 'tags' para conseguir interpretar se evento foi bem sucedido ou não. 'id' = 1801 ou 1802
    events['tags'] = events['tags'].map(utils.converting_tags)

    #fazer o mesmo do df das networks para esse aqui (criar df com features)
    df = matches[['wyId']]
    df = df.rename(columns={'wyId':'matchID'})

    #colocar team1 e team2
    home_l = []
    away_l = []

    for match in matches['teamsData']:
        t0 = list(match.values())[0]
        t1 = list(match.values())[1]

        if t0['side'] == 'home':
            home_l.append(t0['teamId'])
            away_l.append(t1['teamId'])  
        else: 
            away_l.append(t0['teamId'])
            home_l.append(t1['teamId'])

    df['team1_ID'] = home_l
    df['team2_ID'] = away_l

    df['tournament'] = tournament


    # GK_SAVES

    #contar quantos subEvents = 'Save attempt' com tag = 1801(bem sucedido) para todos os (match, team)
    gk_saves = events[(events['eventName'] == 'Save attempt') & (events['tags'].apply(lambda x: 1801 in x))].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
    gk_saves.columns = gk_saves.columns.droplevel(0)
    gk_saves = gk_saves.reset_index().rename(columns={'count':'gk_saves'})

    #juntar feature com df
    df = df.merge(gk_saves, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
    df = df.merge(gk_saves, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

    df = df.rename(columns={"gk_saves_x": "gk_saves_T1", "gk_saves_y": "gk_saves_T2"})


    #RED_CARDS

    #contar quantos subEvents = 'Foul' com tag = 1701(red card) ou 1703(second yellow card) para todos os (match, team)
    red_card = events[events['tags'].apply(lambda x: (1701 in x) or (1703 in x))].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
    red_card.columns = red_card.columns.droplevel(0)
    red_card = red_card.reset_index().rename(columns={'count':'red_card'})

    #juntar feature com df
    df = df.merge(red_card, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
    df = df.merge(red_card, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

    df = df.rename(columns={"red_card_x": "red_card_T1", "red_card_y": "red_card_T2"})


    #YELLOW_CARDS

    #contar quantos subEvents = 'Foul' com tag = 1702(yellow card) ou 1703(second yellow card) para todos os (match, team)
    yellow_card = events[events['tags'].apply(lambda x: (1702 in x) or (1703 in x))].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
    yellow_card.columns = yellow_card.columns.droplevel(0)
    yellow_card = yellow_card.reset_index().rename(columns={'count':'yellow_card'})

    #juntar feature com df
    df = df.merge(yellow_card, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
    df = df.merge(yellow_card, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

    df = df.rename(columns={"yellow_card_x": "yellow_card_T1", "yellow_card_y": "yellow_card_T2"})


    #ASSISTS

    assists = events[events['tags'].apply(lambda x: (301 in x))].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
    assists.columns = assists.columns.droplevel(0)
    assists = assists.reset_index().rename(columns={'count':'assists'})

    #juntar feature com df
    df = df.merge(assists, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
    df = df.merge(assists, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

    df = df.rename(columns={"assists_x": "assists_T1", "assists_y": "assists_T2"})


    #FINALIZAÇÕES

    #contar quantos subEventName = 'Shot', 'Free kick shot' ou 'Penalty' para todos os (match, team)

    shots = events[events['subEventName'].isin(['Shot', 'Free kick shot', 'Penalty'])].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
    shots.columns = shots.columns.droplevel(0)
    shots = shots.reset_index().rename(columns={'count':'shots'})

    #juntar feature com df
    df = df.merge(shots, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
    df = df.merge(shots, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

    df = df.rename(columns={"shots_x": "shots_T1", "shots_y": "shots_T2"})


    #CHUTES A GOL

    #contar quantos subEventName = 'Shot', 'Free kick shot' ou 'Penalty' com tag = 1801(accurate) para todos os (match, team)
    shots_on_target = events[(events['subEventName'].isin(['Shot', 'Free kick shot', 'Penalty'])) & (events['tags'].apply(lambda x: (1801 in x)))].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
    shots_on_target.columns = shots_on_target.columns.droplevel(0)
    shots_on_target = shots_on_target.reset_index().rename(columns={'count':'shots_on_target'})

    #juntar feature com df
    df = df.merge(shots_on_target, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
    df = df.merge(shots_on_target, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

    df = df.rename(columns={"shots_on_target_x": "shots_on_target_T1", "shots_on_target_y": "shots_on_target_T2"})


    #QUANTIDADE DE PASSES

    passes = events[(events['eventName'] == 'Pass') & (events['tags'].apply(lambda x: (1801 in x)))].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
    passes.columns = passes.columns.droplevel(0)
    passes = passes.reset_index().rename(columns={'count':'passes'})

    #juntar feature com df
    df = df.merge(passes, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
    df = df.merge(passes, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

    df = df.rename(columns={"passes_x": "passes_T1", "passes_y": "passes_T2"})


    #GOLS

    goals = events[(events['eventName'] != 'Save attempt') & (events['tags'].apply(lambda x: (101 in x)))].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
    goals.columns = goals.columns.droplevel(0)
    goals = goals.reset_index().rename(columns={'count':'goals'})

    own_goals = events[(events['eventName'] != 'Save attempt') & (events['tags'].apply(lambda x: (102 in x)))].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
    own_goals.columns = own_goals.columns.droplevel(0)
    own_goals = own_goals.reset_index().rename(columns={'count':'own_goals'})

    #juntar feature com df
    df = df.merge(goals, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
    df = df.merge(goals, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

    df = df.merge(own_goals, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
    df = df.merge(own_goals, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

    df = df.rename(columns={"goals_x": "goals_T1", "goals_y": "goals_T2"})
    df = df.rename(columns={"own_goals_x": "own_goals_T1", "own_goals_y": "own_goals_T2"})

    df = df.fillna(0)

    df['goals_T1'] = df['goals_T1'] + df['own_goals_T2']
    df['goals_T2'] = df['goals_T2'] + df['own_goals_T1']

    df = df.drop(['own_goals_T1', 'own_goals_T2'], axis=1)


    #POSSE DE BOLA

    #passes dados / total de passes na partida
    df['ball_possession_T1'] = df['passes_T1'] / (df['passes_T1'] + df['passes_T2'])
    df['ball_possession_T2'] = df['passes_T2'] / (df['passes_T1'] + df['passes_T2'])


    #ACCURACY

    # **pass_accuracy**
    total_passes = events[events['eventName'] == 'Pass'].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
    total_passes.columns = total_passes.columns.droplevel(0)
    total_passes = total_passes.reset_index().rename(columns={'count':'total_passes'})

    #juntar feature com df
    df = df.merge(total_passes, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
    df = df.merge(total_passes, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

    df = df.rename(columns={"total_passes_x": "total_passes_T1", "total_passes_y": "total_passes_T2"})

    df['pass_acc_T1'] = df['passes_T1'] / df['total_passes_T1']
    df['pass_acc_T2'] = df['passes_T2'] / df['total_passes_T2']

    # **gk_accuracy**
    gk_att = events[(events['eventName'] == 'Save attempt')].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
    gk_att.columns = gk_att.columns.droplevel(0)
    gk_att = gk_att.reset_index().rename(columns={'count':'gk_att'})

    #juntar feature com df
    df = df.merge(gk_att, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
    df = df.merge(gk_att, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

    df = df.rename(columns={"gk_att_x": "gk_att_T1", "gk_att_y": "gk_att_T2"})

    df['gk_acc_T1'] = df['gk_saves_T1'] / df['gk_att_T1']
    df['gk_acc_T2'] = df['gk_saves_T2'] / df['gk_att_T2']

    # **shot_accuracy**
    df['shot_acc_T1'] = df['shots_on_target_T1'] / df['shots_T1']
    df['shot_acc_T2'] = df['shots_on_target_T2'] / df['shots_T2']

    df = df.drop(['total_passes_T1', 'total_passes_T2', 'gk_att_T1', 'gk_att_T2'], axis=1)
    df = df.fillna(0)


    #WIN COLUMN
    df['winner'] = np.where((matches['winner'] == df['team1_ID']), 1, 0)

In [9]:
df

,matchID,team1_ID,team2_ID,gk_saves_T1,gk_saves_T2,red_card_T1,red_card_T2,yellow_card_T1,yellow_card_T2,assists_T1,assists_T2,shots_T1,shots_T2,shots_on_target_T1,shots_on_target_T2,passes_T1,passes_T2,goals_T1,goals_T2,ball_possession_T1,ball_possession_T2,pass_acc_T1,pass_acc_T2,gk_acc_T1,gk_acc_T2,shot_acc_T1,shot_acc_T2,winner
0,2058017,4418,9598,1.0,3.0,0.0,0.0,2.0,1.0,0.0,1.0,8,14,6.0,3.0,186,429,4.0,2.0,0.302439,0.697561,0.771784,0.828185,0.333333,0.428571,0.750000,0.214286,1
1,2058016,5629,2413,6.0,2.0,0.0,0.0,1.0,2.0,2.0,0.0,11,14,4.0,5.0,427,593,2.0,0.0,0.418627,0.581373,0.880412,0.906728,1.000000,0.500000,0.363636,0.357143,1
2,2058015,9598,2413,1.0,5.0,0.0,0.0,2.0,1.0,2.0,0.0,22,11,7.0,2.0,470,370,2.0,1.0,0.559524,0.440476,0.789916,0.825893,0.500000,0.714286,0.318182,0.181818,1
3,2058014,4418,5629,2.0,4.0,0.0,0.0,2.0,3.0,1.0,0.0,18,9,5.0,2.0,282,535,1.0,0.0,0.345165,0.654835,0.889590,0.905245,1.000000,0.800000,0.277778,0.222222,1
4,2058012,14358,9598,1.0,5.0,0.0,0.0,1.0,4.0,1.0,2.0,11,17,6.0,3.0,274,594,2.0,2.0,0.315668,0.684332,0.754821,0.830769,0.333333,0.714286,0.545455,0.176471,0
5,2058013,7047,2413,0.0,3.0,0.0,0.0,2.0,1.0,0.0,2.0,7,11,3.0,2.0,280,402,0.0,2.0,0.410557,0.589443,0.811594,0.828866,0.000000,1.000000,0.428571,0.181818,0
6,2058011,6380,5629,2.0,8.0,0.0,0.0,2.0,2.0,1.0,1.0,25,8,9.0,3.0,437,304,1.0,2.0,0.589744,0.410256,0.877510,0.873563,0.500000,0.888889,0.360000,0.375000,0
7,2058010,15670,4418,0.0,3.0,0.0,0.0,2.0,2.0,0.0,2.0,9,9,3.0,2.0,227,417,0.0,2.0,0.352484,0.647516,0.746711,0.817647,0.000000,1.000000,0.333333,0.222222,0
8,2058009,12430,2413,1.0,3.0,0.0,0.0,6.0,2.0,1.0,0.0,13,15,4.0,2.0,406,457,1.0,1.0,0.470452,0.529548,0.816901,0.863894,0.500000,0.750000,0.307692,0.133333,0
9,2058008,7047,6697,4.0,2.0,0.0,1.0,1.0,2.0,1.0,0.0,12,17,3.0,4.0,192,476,1.0,0.0,0.287425,0.712575,0.777328,0.853047,1.000000,0.666667,0.250000,0.235294,1
